# Часть 1. Кластеризация текста

Необходимо понять относится ли текст к заданным тематикам или нет

#### Подключаем библиотеки

In [1]:
import pandas as pd
import numpy as np
import re
import string
from tqdm.auto import tqdm, trange


from nltk.stem.snowball import SnowballStemmer
from nltk import word_tokenize
import nltk
import pymorphy2

from spacy.lang.ru import Russian
from spacy.matcher import PhraseMatcher
import spacy

from wordcloud import WordCloud

import matplotlib.pyplot as plt

import warnings
warnings.filterwarnings("ignore")

#### На вход получаем csv файл с текстом и ДОПИСАТЬ

In [2]:
url='https://drive.google.com/file/d/1lQMb5B_obFaOzmPxtVEVizT-Fe0EcTao/view?usp=sharing'
url='https://drive.google.com/uc?id=' + url.split('/')[-2]
df_news = pd.read_csv(url)

url_check='https://drive.google.com/file/d/1RITlCpMRS8NCQHLZ-cE8VBDGUCCw_XBz/view?usp=sharing'
url_check ='https://drive.google.com/uc?id=' + url_check.split('/')[-2]
check = pd.read_csv(url_check)

In [3]:
df_news

,text,topic
0,Финалисты школьного аксилератора Сбера смогут ...,Другое
1,Администрация президента США Джо Байдена плани...,Другое
2,Совет Федерации одобрили закон о предоставлени...,Другое
3,Россиянам осенью доступны заграничные турпакет...,Другое
4,Россияне пожаловались на «баснословный» рост с...,Другое
5,Редакторы модного журнала Grazia раскрыли спос...,Другое
6,"Американский рэпер и дизайнер Канье Уэст, кото...",Другое
7,Редакторы журнала Men Today перечислили детали...,Другое
8,Лауреатами Нобелевской премии по физике за 202...,Другое
9,В штате Небраска (США) смартфон Apple самостоя...,Другое


## Стемминг

In [4]:
stemmer = SnowballStemmer("russian")

def get_token(text):
    tokens = [word for sent in nltk.sent_tokenize(text) for word in nltk.word_tokenize(sent)]
    filtered_tokens = []
    for token in tokens:
        if re.search('[а-яА-Я]', token):
            filtered_tokens.append(token)
    stems = [stemmer.stem(t) for t in filtered_tokens]
    return stems

## Создаем список стоп-слов

In [5]:
stopwords = nltk.corpus.stopwords.words('russian')

stopwords.extend(['что', 'так', 'вот', 'быть', 'как', 'в', 'к', 'на', 'это', "эта", 'нею', "этих", 'которые'
                 ,'довольно', 'нам', 'самом', "сразу", "таких", "такое", "такие", "такую", "таким", "этим"
                 ,"другими", "могут", "изза", "пока", "которой", "несмотря", "которых", "который", "своя"
                 , "наш", "самый", "например", "нужно", "просто", "вообще", "очень", "сами", "своей"
                 , "какоето","гдето", "никак", "както", "которого", "хотя", "тех", "наша", 'всё', 'другие'])

## Токенизация текстов

In [6]:
def text_to_tokens (text): 
    text = text.lower()
    spec_chars = string.punctuation + '\n\xa0«»\t—…'
    text = "".join([ch for ch in text if ch not in spec_chars])
    text = re.sub('\n', '', text)
    text = remove_chars_from_text(text, spec_chars)
    text = remove_chars_from_text(text, string.digits)
    text_tokens = word_tokenize(text)
    
    morph = pymorphy2.MorphAnalyzer()
    for i in range(len(text_tokens)):
        text_tokens[i] = morph.parse(text_tokens[i])[0].normal_form
        
    text = nltk.Text(text_tokens)
    text_tokens = [token.strip() for token in text_tokens if token not in stopwords]
    return text_tokens

In [7]:
def remove_chars_from_text(text, chars):
    return "".join([ch for ch in text if ch not in chars])


In [8]:
tqdm.pandas()
df_news['text_token'] = df_news['text'].progress_apply(lambda x : text_to_tokens(x))



  0%|          | 0/38 [00:00<?, ?it/s]

## Проводим кластеризацию текстов по темам


In [9]:
nlp = spacy.load('ru_core_news_lg') 
phrase_matcher = PhraseMatcher(nlp.vocab)

phrases = list(check['Дир'].dropna())
patterns = [nlp(text) for text in phrases]

phrase_matcher.add('AI', None, *patterns)

In [10]:
#!pip install -U pip setuptools wheel
#!pip install -U spacy
#!python -m spacy download en_core_web_sm
#!python -m spacy download ru_core_news_lg

In [11]:
filter_buh = list(check['Бух'])
filter_hr = list(check['HR'])
filter_SPAM = list(check['Анти'])



In [12]:
df_news['buh_points'] = 0
df_news['hr_points'] = 0
df_news['dir_points'] = 0

for i in range(len(df_news)):
    
    #очки для новостей для директора
    text= str.lower(df_news['text'][i])
    sentence = nlp (text) 
    matched_phrases = phrase_matcher(sentence)
    df_news['dir_points'][i] = len(matched_phrases)
    
    for token in df_news['text_token'][i]:
        if token in filter_buh:
            df_news['buh_points'][i] +=1
        if token in filter_hr:
            df_news['hr_points'][i] +=1
        if token in filter_SPAM:
            df_news['hr_points'][i] -=1
            df_news['buh_points'][i] -=1
        

## Пояснение 

Если в тексте больше 4 балов в столбце buh_points, hr_points или dir_points значит текст можно отнести к категории "Бухгалтер", "HR", "Директор" соответтсвенно

In [13]:
df_news


,text,topic,text_token,buh_points,hr_points,dir_points
0,Финалисты школьного аксилератора Сбера смогут ...,Другое,"[финалист, школьный, аксилератор, сбер, смочь,...",0,0,0
1,Администрация президента США Джо Байдена плани...,Другое,"[администрация, президент, сша, джо, байден, п...",-2,-2,0
2,Совет Федерации одобрили закон о предоставлени...,Другое,"[совет, федерация, одобрить, закон, предоставл...",1,0,1
3,Россиянам осенью доступны заграничные турпакет...,Другое,"[россиянин, осень, доступный, заграничный, тур...",0,0,0
4,Россияне пожаловались на «баснословный» рост с...,Другое,"[россиянин, пожаловаться, баснословный, рост, ...",2,-1,0
5,Редакторы модного журнала Grazia раскрыли спос...,Другое,"[редактор, модный, журнал, grazia, раскрыть, с...",0,1,0
6,"Американский рэпер и дизайнер Канье Уэст, кото...",Другое,"[американский, рэпер, дизайнер, канье, уэст, о...",0,0,0
7,Редакторы журнала Men Today перечислили детали...,Другое,"[редактор, журнал, men, today, перечислить, де...",0,1,0
8,Лауреатами Нобелевской премии по физике за 202...,Другое,"[лауреат, нобелевский, премия, физика, год, ст...",0,0,0
9,В штате Небраска (США) смартфон Apple самостоя...,Другое,"[штат, небраска, сша, смартфон, apple, самосто...",-1,-1,0


Ниже отбираются только темы где больше 4 балов

In [14]:
df_news = df_news[(df_news['buh_points']>4) | (df_news['hr_points']>4) | (df_news['dir_points']>2)].reset_index(drop=True)

df_news

,text,topic,text_token,buh_points,hr_points,dir_points
0,Если взглянуть на тренды поисковых запросов в ...,HR,"[взглянуть, тренд, поисковый, запрос, google, ...",0,6,0
1,Аналитики IT-холдинга TalentTech совместно с п...,HR,"[аналитик, itхолдинг, talenttech, совместно, п...",8,23,1
2,Специальные алгоритмы платформы активируются п...,HR,"[специальный, алгоритм, платформа, активироват...",8,20,0
3,"Рынок труда — зеркало того, что происходит на ...",HR,"[рынок, труд, зеркало, происходить, рынок, цел...",2,5,0
4,Необходимость во временных сотрудниках возника...,HR,"[необходимость, временной, сотрудник, возникат...",1,13,0
5,Порой мы зацикливаемся на трудоустройстве рядо...,HR,"[порой, зацикливаться, трудоустройство, рядом,...",1,9,0
6,Как изменился рынок труда за неделю? В каких с...,HR,"[измениться, рынок, труд, неделя, сфера, регио...",0,10,0
7,"Практика современных компаний показывает, что ...",HR,"[практика, современный, компания, показывать, ...",0,5,0
8,"ванию. Бухгалтеры, как и раньше, руководствуют...",Бухгалтерия,"[вания, бухгалтер, ранний, руководствоваться, ...",5,1,0
9,Что еще за БухДжоб\nБухДжоб — топовый канал «К...,Бухгалтерия,"[ещё, бухджоббухджоб, топовый, канал, клерк, т...",11,5,0


In [15]:
df_news['buh_final'] = 0
df_news['hr_final'] = 0
df_news['dir_final'] = 0

for i in range(len(df_news)):
    if df_news['buh_points'][i] >4:
        df_news['buh_final'][i] = 1
    if df_news['hr_points'][i] >4:
        df_news['hr_final'][i] = 1
    if df_news['dir_points'][i] >4:
        df_news['dir_final'][i] = 1
        

In [16]:
df_news

,text,topic,text_token,buh_points,hr_points,dir_points,buh_final,hr_final,dir_final
0,Если взглянуть на тренды поисковых запросов в ...,HR,"[взглянуть, тренд, поисковый, запрос, google, ...",0,6,0,0,1,0
1,Аналитики IT-холдинга TalentTech совместно с п...,HR,"[аналитик, itхолдинг, talenttech, совместно, п...",8,23,1,1,1,0
2,Специальные алгоритмы платформы активируются п...,HR,"[специальный, алгоритм, платформа, активироват...",8,20,0,1,1,0
3,"Рынок труда — зеркало того, что происходит на ...",HR,"[рынок, труд, зеркало, происходить, рынок, цел...",2,5,0,0,1,0
4,Необходимость во временных сотрудниках возника...,HR,"[необходимость, временной, сотрудник, возникат...",1,13,0,0,1,0
5,Порой мы зацикливаемся на трудоустройстве рядо...,HR,"[порой, зацикливаться, трудоустройство, рядом,...",1,9,0,0,1,0
6,Как изменился рынок труда за неделю? В каких с...,HR,"[измениться, рынок, труд, неделя, сфера, регио...",0,10,0,0,1,0
7,"Практика современных компаний показывает, что ...",HR,"[практика, современный, компания, показывать, ...",0,5,0,0,1,0
8,"ванию. Бухгалтеры, как и раньше, руководствуют...",Бухгалтерия,"[вания, бухгалтер, ранний, руководствоваться, ...",5,1,0,1,0,0
9,Что еще за БухДжоб\nБухДжоб — топовый канал «К...,Бухгалтерия,"[ещё, бухджоббухджоб, топовый, канал, клерк, т...",11,5,0,1,1,0


## Получаем финальную таблицу с текстом, токенизированным текстом и распределением по темам

In [17]:
df_news = df_news[['text', 'text_token', 'topic', 'buh_final', 'hr_final', 'dir_final']]
df_news

,text,text_token,topic,buh_final,hr_final,dir_final
0,Если взглянуть на тренды поисковых запросов в ...,"[взглянуть, тренд, поисковый, запрос, google, ...",HR,0,1,0
1,Аналитики IT-холдинга TalentTech совместно с п...,"[аналитик, itхолдинг, talenttech, совместно, п...",HR,1,1,0
2,Специальные алгоритмы платформы активируются п...,"[специальный, алгоритм, платформа, активироват...",HR,1,1,0
3,"Рынок труда — зеркало того, что происходит на ...","[рынок, труд, зеркало, происходить, рынок, цел...",HR,0,1,0
4,Необходимость во временных сотрудниках возника...,"[необходимость, временной, сотрудник, возникат...",HR,0,1,0
5,Порой мы зацикливаемся на трудоустройстве рядо...,"[порой, зацикливаться, трудоустройство, рядом,...",HR,0,1,0
6,Как изменился рынок труда за неделю? В каких с...,"[измениться, рынок, труд, неделя, сфера, регио...",HR,0,1,0
7,"Практика современных компаний показывает, что ...","[практика, современный, компания, показывать, ...",HR,0,1,0
8,"ванию. Бухгалтеры, как и раньше, руководствуют...","[вания, бухгалтер, ранний, руководствоваться, ...",Бухгалтерия,1,0,0
9,Что еще за БухДжоб\nБухДжоб — топовый канал «К...,"[ещё, бухджоббухджоб, топовый, канал, клерк, т...",Бухгалтерия,1,1,0


 # Часть 2. Отбор основного содержания

In [18]:
from nltk.probability import FreqDist

In [19]:
def find_most_popular(list_sentences, df_sort_10):
    list_most_pop = []
    for d_s in range(len(list_sentences)):
        list1 = []
        for pop_word in df_sort_10['Слова']:
            if pop_word in list_sentences[d_s]: # если самые популярные слова есть в предложении
                list1.append(d_s)               # то в список добавляем ID предложения?
        list_most_pop.append(list1)             # в итоге получаем list1 в котором n количество раз написан ID предложения 
                                                # добавляем этот список
    return list_most_pop                    

def make_df_with_sent_num(list_most_pop):
    list_new_new = []
    list_usefull_word_count = []
    for i in list_most_pop:
        if i!=[]:
            list_new_new.append(i[0])
            list_usefull_word_count.append(len(i))   
            
    df_final = pd.DataFrame({'sentence_number':list_new_new,
                     'usefull_word_count':list_usefull_word_count})
    df_final = df_final.sort_values('usefull_word_count', ascending=False).head(3)
    df_final = df_final.sort_values('sentence_number')
    
    return df_final

In [20]:
df_news['main_sence'] = 0

for i in range(len(df_news)):
    text = nltk.Text(df_news['text_token'][i])
    fdist_sw = FreqDist(text)
    most_common_10 = fdist_sw.most_common(10)
    df_sort_10 = pd.DataFrame({'dd':fdist_sw})
    df_sort_10 = df_sort_10.reset_index().rename(columns={"index": "Слова", "dd": "Встречаемость"},  inplace=False)
    df_sort_10 = df_sort_10.sort_values('Встречаемость', ascending=False).head(10)
    text_all = df_news['text'][i]
    list_sentences = text_all.split('.')
    
    
    count = 0
    dict_senteces = {}
    for sentence in list_sentences:
        if sentence !='': # все предложения пронумировуем и добавляем в словарь. Хотя можно было тоже самое сделать со списком
            dict_senteces[count] = sentence
            count+=1
            
    list_most_pop = find_most_popular(list_sentences, df_sort_10)
    
    df_final = make_df_with_sent_num(list_most_pop)
    
    str_main = ''
    for j in dict_senteces:
        if j in df_final['sentence_number']:
            str_main += dict_senteces[j] + '.'
    df_news['main_sence'][i] = str_main


In [21]:
df_news

,text,text_token,topic,buh_final,hr_final,dir_final,main_sence
0,Если взглянуть на тренды поисковых запросов в ...,"[взглянуть, тренд, поисковый, запрос, google, ...",HR,0,1,0,"Должность, связывающая бизнес и HR, сначала п..."
1,Аналитики IT-холдинга TalentTech совместно с п...,"[аналитик, itхолдинг, talenttech, совместно, п...",HR,1,1,0,Аналитики IT-холдинга TalentTech совместно с п...
2,Специальные алгоритмы платформы активируются п...,"[специальный, алгоритм, платформа, активироват...",HR,1,1,0,Специальные алгоритмы платформы активируются п...
3,"Рынок труда — зеркало того, что происходит на ...","[рынок, труд, зеркало, происходить, рынок, цел...",HR,0,1,0,С марта 2020 года пандемия коронавируса оказы...
4,Необходимость во временных сотрудниках возника...,"[необходимость, временной, сотрудник, возникат...",HR,0,1,0,Необходимость во временных сотрудниках возника...
5,Порой мы зацикливаемся на трудоустройстве рядо...,"[порой, зацикливаться, трудоустройство, рядом,...",HR,0,1,0,Порой мы зацикливаемся на трудоустройстве рядо...
6,Как изменился рынок труда за неделю? В каких с...,"[измениться, рынок, труд, неделя, сфера, регио...",HR,0,1,0,Как изменился рынок труда за неделю? В каких с...
7,"Практика современных компаний показывает, что ...","[практика, современный, компания, показывать, ...",HR,0,1,0,"Практика современных компаний показывает, что ..."
8,"ванию. Бухгалтеры, как и раньше, руководствуют...","[вания, бухгалтер, ранний, руководствоваться, ...",Бухгалтерия,1,0,0,"5-ж, п. Капитальные вложения должны изменять ..."
9,Что еще за БухДжоб\nБухДжоб — топовый канал «К...,"[ещё, бухджоббухджоб, топовый, канал, клерк, т...",Бухгалтерия,1,1,0,Что еще за БухДжоб\nБухДжоб — топовый канал «К...


## Ранжирование новости по важности

Будем ранжировать при помощи n-грамм-маячков.
Если в новости встречаются слова "штраф", "предусмотренна ответственность", "запрещенно" и т.д. новости в которых рассказывается про какую-то ответственность и ограничения, то это самые важные новости.
На втором месте новости со словами "субсидии", "законы", "постановления" и т.д. то есть новости о каких-то изменениях или "плюшках".
Все остальное в третий тип

### Выделяем новости с рейтингом А

In [22]:
phrases_A = list(check['Рейтинг A'].dropna())
patterns_A = [nlp(text) for text in phrases_A]
phrase_matcher.add('AI', None, *patterns_A)
df_news['rating_A'] = 0
df_news['rating_B'] = 0
df_news['rating_C'] = 1


for i in range(len(df_news)):
    #print(df_news['text'][i])
    text = str.lower(df_news['text'][i])
    sentence = nlp (text) 
    matched_phrases = phrase_matcher(sentence)
    df_news['rating_A'][i] = len(matched_phrases)        

In [23]:
phrases_B = list(check['Рейтинг B'].dropna())
patterns_B = [nlp(text) for text in phrases_B]
phrase_matcher.add('AI', None, *patterns_B)

for i in range(len(df_news)):
    #print(df_news['text'][i])
    text = str.lower(df_news['text'][i])
    sentence = nlp (text) 
    matched_phrases = phrase_matcher(sentence)
    df_news['rating_B'][i] = len(matched_phrases)  

In [24]:
df_news

,text,text_token,topic,buh_final,hr_final,dir_final,main_sence,rating_A,rating_B,rating_C
0,Если взглянуть на тренды поисковых запросов в ...,"[взглянуть, тренд, поисковый, запрос, google, ...",HR,0,1,0,"Должность, связывающая бизнес и HR, сначала п...",0,0,1
1,Аналитики IT-холдинга TalentTech совместно с п...,"[аналитик, itхолдинг, talenttech, совместно, п...",HR,1,1,0,Аналитики IT-холдинга TalentTech совместно с п...,1,3,1
2,Специальные алгоритмы платформы активируются п...,"[специальный, алгоритм, платформа, активироват...",HR,1,1,0,Специальные алгоритмы платформы активируются п...,0,0,1
3,"Рынок труда — зеркало того, что происходит на ...","[рынок, труд, зеркало, происходить, рынок, цел...",HR,0,1,0,С марта 2020 года пандемия коронавируса оказы...,0,0,1
4,Необходимость во временных сотрудниках возника...,"[необходимость, временной, сотрудник, возникат...",HR,0,1,0,Необходимость во временных сотрудниках возника...,0,0,1
5,Порой мы зацикливаемся на трудоустройстве рядо...,"[порой, зацикливаться, трудоустройство, рядом,...",HR,0,1,0,Порой мы зацикливаемся на трудоустройстве рядо...,0,0,1
6,Как изменился рынок труда за неделю? В каких с...,"[измениться, рынок, труд, неделя, сфера, регио...",HR,0,1,0,Как изменился рынок труда за неделю? В каких с...,0,0,1
7,"Практика современных компаний показывает, что ...","[практика, современный, компания, показывать, ...",HR,0,1,0,"Практика современных компаний показывает, что ...",0,0,1
8,"ванию. Бухгалтеры, как и раньше, руководствуют...","[вания, бухгалтер, ранний, руководствоваться, ...",Бухгалтерия,1,0,0,"5-ж, п. Капитальные вложения должны изменять ...",0,0,1
9,Что еще за БухДжоб\nБухДжоб — топовый канал «К...,"[ещё, бухджоббухджоб, топовый, канал, клерк, т...",Бухгалтерия,1,1,0,Что еще за БухДжоб\nБухДжоб — топовый канал «К...,0,0,1


## Поиск трендов

Сбросим все токены за конкретный период в один список.
А далее построим карту слов и частотности.


In [25]:
all_tokens = []
pairs_tokens = []
for tokens in df_news['text_token']:
    #if date== date тут формула для выбора даты
    all_tokens.extend(tokens)

In [26]:
stopwords.extend(["также", "ваш", "свой", "весь", "изз", "обычно"])

In [27]:
all_tokens = [token.strip() for token in all_tokens if token not in stopwords]


In [28]:
for i in range(len(all_tokens)-1):
    pairs_tokens.append(str(all_tokens[i] + " " + all_tokens[i+1]))


In [29]:
from nltk.probability import FreqDist
fdist = FreqDist(pairs_tokens)
fdist

FreqDist({'авить работа': 8, 'рынок труд': 6, 'основный средство': 5, 'приказ минфин': 5, 'минфин №': 5, 'п пбу': 5, 'получение аккредитация': 5, 'профильный код': 5, 'управленческий учёт': 5, 'стоимость актив': 4, ...})